In [85]:
import pandas as pd 
from datetime import datetime, timedelta, date, time

df = pd.read_csv("data.csv")

# Convert times to datetime.time objects
df['arrive_time_obj'] = pd.to_datetime(df['arrive_time'], format='%H:%M').dt.time
df['leave_time_obj'] = pd.to_datetime(df['leave_time'], format='%H:%M').dt.time



def best_leave_time(target_time):

    target_datetime = datetime.strptime(target_time, '%H:%M')
    target_time = datetime.strptime(target_time, '%H:%M').time()

    df_filtered = df[df['arrive_time_obj'] <= target_time]

    
    idx_latest = df_filtered.groupby('day_of_week')['leave_time_obj'].idxmax()

    # Extract rows with both leave_time and duration_minutes
    best_leave_times = df_filtered.loc[idx_latest, ['day_of_week', 'leave_time_obj', 'duration_minutes']]

    # Sort day_of_week
    day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    best_leave_times['day_of_week'] = pd.Categorical(
        best_leave_times['day_of_week'],
        categories=day_order,
        ordered=True
    )
    best_leave_times = best_leave_times.sort_values('day_of_week')

    best_leave_times['duration_minutes'] = best_leave_times['duration_minutes'].round().astype(int)
    best_leave_times[f'leave_time_to_arrive_by_{target_time}'] = best_leave_times['duration_minutes'].apply(lambda x: (target_datetime - timedelta(minutes=x)).strftime('%H:%M'))

    best_leave_times = best_leave_times.drop(columns=['leave_time_obj'])
    best_leave_times = best_leave_times.iloc[:,[0,2,1]]
    print(best_leave_times.to_string(index=False))


In [86]:
# target_time = input('When do u want to arrive (HH:MM) :')
best_leave_time('9:30')
# result

day_of_week leave_time_to_arrive_by_09:30:00  duration_minutes
     Monday                            08:36                54
    Tuesday                            08:35                55
  Wednesday                            08:30                60
   Thursday                            08:35                55
     Friday                            08:47                43
